# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [12]:
# Write your code below.
%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [13]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [17]:
import os
from glob import glob

# Write your code below.

# Load environment variable
PRICE_DATA = os.getenv("PRICE_DATA")

# Use glob to find all parquet files recursively under PRICE_DATA
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)

print("Found parquet files:", len(parquet_files))
print(parquet_files[:5])  # peek at first few


Found parquet files: 2940
['../../05_src/data/prices/JHMF/JHMF_2015/part.0.parquet', '../../05_src/data/prices/JHMF/JHMF_2015/part.1.parquet', '../../05_src/data/prices/JHMF/JHMF_2019/part.0.parquet', '../../05_src/data/prices/JHMF/JHMF_2019/part.1.parquet', '../../05_src/data/prices/JHMF/JHMF_2017/part.0.parquet']


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [19]:
# Write your code below.

# Read into a Dask DataFrame
dd_px = dd.read_parquet(parquet_files, engine="pyarrow")

# Normalize column name for adjusted close if needed
if "Adj Close" in dd_px.columns:
    dd_px = dd_px.rename(columns={"Adj Close": "Adj_Close"})

# Ensure ticker is a column (not index)
if "ticker" not in dd_px.columns:
    dd_px = dd_px.reset_index()

# --- Build features per ticker ---
def _feat_per_ticker(pdf):
    # Sort within each ticker by Date to make shifts/returns correct
    pdf = pdf.sort_values("Date")
    # Lags
    pdf["Close_lag_1"] = pdf["Close"].shift(1)
    pdf["Adj_Close_lag_1"] = pdf["Adj_Close"].shift(1)
    # Returns
    pdf["returns"] = pdf["Close"] / pdf["Close_lag_1"] - 1
    # Range (High - Low)
    pdf["hi_lo_range"] = pdf["High"] - pdf["Low"]
    # Keep useful columns
    return pdf[[
        "ticker","Date","Open","High","Low","Close","Adj_Close","Volume",
        "Close_lag_1","Adj_Close_lag_1","returns","hi_lo_range"
    ]]

meta = {
    "ticker": "object",
    "Date": "datetime64[ns]",
    "Open": "f8", "High": "f8", "Low": "f8", "Close": "f8", "Adj_Close": "f8",
    "Volume": "f8",
    "Close_lag_1": "f8", "Adj_Close_lag_1": "f8",
    "returns": "f8", "hi_lo_range": "f8",
}

dd_feat = (
    dd_px.groupby("ticker", group_keys=False)
         .apply(_feat_per_ticker, meta=meta)
)

dd_feat.head()


,ticker,Date,Open,High,Low,Close,Adj_Close,Volume,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [20]:
# Write your code below.
# 1) Convert Dask DataFrame to pandas
df_feat = dd_feat.compute()

# 2) Sort so rolling works properly within each ticker
df_feat = df_feat.sort_values(["ticker", "Date"])

# 3) Compute 10-day moving average of returns per ticker
df_feat["ma_returns_10"] = (
    df_feat.groupby("ticker")["returns"]
           .transform(lambda x: x.rolling(10).mean())
)

# 4) Inspect results
print(df_feat.head(15))


      ticker       Date       Open       High        Low      Close  \
38739      A 1999-11-18  32.546494  35.765381  28.612303  31.473534   
38740      A 1999-11-19  30.713520  30.758226  28.478184  28.880543   
38741      A 1999-11-22  29.551144  31.473534  28.657009  31.473534   
38742      A 1999-11-23  30.400572  31.205294  28.612303  28.612303   
38743      A 1999-11-24  28.701717  29.998211  28.612303  29.372318   
38744      A 1999-11-26  29.238197  29.685265  29.148785  29.461731   
38745      A 1999-11-29  29.327610  30.355865  29.014664  30.132332   
38746      A 1999-11-30  30.042919  30.713520  29.282904  30.177038   
38747      A 1999-12-01  30.177038  31.071173  29.953505  30.713520   
38748      A 1999-12-02  31.294706  32.188843  30.892345  31.562946   
38749      A 1999-12-03  32.144135  32.680614  31.697067  31.831188   
38750      A 1999-12-06  32.367668  33.217094  32.322960  32.725323   
38751      A 1999-12-07  32.725323  32.904148  31.697067  32.367668   
38752 

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return? No,   converting to pandas was not strictly necessary — Dask also supports rolling operations with groupby.We used pandas here mainly for simplicity and inspection, while Dask is better suited if the dataset is too large for memory.

+ Would it have been better to do it in Dask? Why? Yes — doing it in Dask is better for large datasets because it processes data in parallel avoiding memory overload. Pandas is fine for smaller datasets, but it forces all data into memory, which doesn’t scale well.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.